In [1]:
import numpy as np
import pandas as pd
import os
import json
import sys

In [2]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('np', 80), ('pd', 80)]

In [6]:
dir_path = '../data/tweets_maga'
data = pd.DataFrame()

print('Loading tweets from', dir_path)
file_names = [file for file in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, file))]
for file_name in file_names:
    file_path = dir_path + '/' + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        tweets = pd.read_json(file)
        data = pd.concat([data, tweets])
        
print('Successfully loaded {} tweets from {} files.'.format(len(data), len(file_names)))

Loading tweets from ../data/tweets_maga
Successfully loaded 14000 tweets from 14 files.


In [7]:
data.head()

,author_name,created_at,id,text
0,RealPatriot,2019-10-01 00:37:21,1178831229963116544,@realDonaldTrump I’m confused. Is this a new “...
1,Teresa murray⭐⭐⭐ CIA #fakewistleblower 😡,2019-10-01 00:37:20,1178831224191832067,@realDonaldTrump @Hyundai @Kia @Aptiv I'm not ...
2,GOPKnowsBest,2019-10-01 00:37:17,1178831213571629057,@WA_Silenced_Maj @realDonaldTrump You are the ...
3,Peter Dunbar,2019-10-01 00:37:15,1178831204151226368,It is amazing to watch @RudyGiuliani throw his...
4,AtlanticCity Deplorable,2019-10-01 00:37:04,1178831160635514880,@w_terrence @realDonaldTrump WHOOOOOHOOOOO! I’...
